# Welcome To the TensorRT Quantization Tutorial!

This notebook is designed to show the features of the TensorRT passes integrated into MASE.

## Section 1
Firstly, we will show you how to do a INT8 quantization of a simple model, `jsc-toy`, and compare the quantized model to the original model. 

We start by loading in the required libraries and passes required for the notebook as well as ensuring the correct path is set for machop to be used.

In [1]:
import sys
import os
from pathlib import Path
import toml
from copy import copy, deepcopy

# Figure out the correct path
machop_path = Path(".").resolve().parent.parent.parent /"machop"
assert machop_path.exists(), "Failed to find machop at: {}".format(machop_path)
sys.path.append(str(machop_path))

# Add directory to the PATH so that chop can be called
new_path = "../../../machop"
full_path = os.path.abspath(new_path)
os.environ['PATH'] += os.pathsep + full_path

from chop.tools.logger import set_logging_verbosity
from chop.passes.graph.utils import deepcopy_mase_graph
from chop.tools.get_input import InputGenerator
from chop.tools.checkpoint_load import load_model
from chop.ir import MaseGraph
from chop.models import get_model_info, get_model
from chop.dataset import MaseDataModule, get_dataset_info
from chop.passes.graph import (
    save_node_meta_param_interface_pass,
    report_node_meta_param_analysis_pass,
    profile_statistics_analysis_pass,
    add_common_metadata_analysis_pass,
    init_metadata_analysis_pass,
    add_software_metadata_analysis_pass,
    tensorrt_calibrate_transform_pass,
    tensorrt_fake_quantize_transform_pass,
    tensorrt_fine_tune_transform_pass,
    tensorrt_engine_interface_pass,
    tensorrt_analysis_pass,
    )

set_logging_verbosity("info")

/opt/conda/envs/mase/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2024-03-16 13:46:41,850] [INFO] [real_accelerator.py:191:get_accelerator] Setting ds_accelerator to cuda (auto detect)


INFO     Set logging level to info
I0316 13:46:43.832612 140639127025472 logger.py:44] Set logging level to info


Next, we load in the toml file used for quantization. To view the configuration, click [here](../../machop/configs/tensorrt/jsc_toy_INT8_quantization_by_type.toml), or read the documentation on Mase [here]().

In [2]:
# Path to your TOML file
toml_file_path = '../../../machop/configs/tensorRT/jsc_toy_INT8_quantization_by_type.toml'

# Reading TOML file and converting it into a Python dictionary
with open(toml_file_path, 'r') as toml_file:
    pass_args = toml.load(toml_file)

# Extract the 'passes.tensorrt_quantize' section and its children
tensorrt_quantize_config = pass_args.get('passes', {}).get('tensorrt_quantize', {})
# Extract the 'passes.tensorrt_fine_tune' section and its children
tensorrt_train_config = pass_args.get('passes', {}).get('tensorrt_fine_tune', {})
# Extract the 'passes.tensorrt_analysis' section and its children
tensorrt_analysis_config = pass_args.get('passes', {}).get('tensorrt_analysis', {})

We then create a `MaseGraph` by loading in a pre-trained model using the checkpoint provided and using the toml configuration model arguments

In [3]:
CHECKPOINT_PATH = "checkpoints/jsc-toy_classification_jsc/software/training_ckpts/best.ckpt"

# Load the basics in to 
model_name = pass_args['model']
dataset_name = pass_args['dataset']
max_epochs = pass_args['max_epochs']
batch_size = pass_args['batch_size']
learning_rate = pass_args['learning_rate']
accelerator = pass_args['accelerator']

data_module = MaseDataModule(
    name=dataset_name,
    batch_size=batch_size,
    model_name=model_name,
    num_workers=0,
)
data_module.prepare_data()
data_module.setup()


model_info = get_model_info(model_name)
# quant_modules.initialize()
model = get_model(
    model_name,
    task="cls",
    dataset_info=data_module.dataset_info,
    pretrained=False)

model = load_model(load_name=CHECKPOINT_PATH, load_type="pl", model=model)

input_generator = InputGenerator(
    data_module=data_module,
    model_info=model_info,
    task="cls",
    which_dataloader="train",
)

dummy_in = next(iter(input_generator))
_ = model(**dummy_in)

# generate the mase graph and initialize node metadata
mg = MaseGraph(model=model)

mg, _ = init_metadata_analysis_pass(mg, None)
mg, _ = add_common_metadata_analysis_pass(mg, {"dummy_in": dummy_in})
mg, _ = add_software_metadata_analysis_pass(mg, None)

INFO     Loaded pytorch lightning checkpoint from checkpoints/jsc-toy_classification_jsc/software/training_ckpts/best.ckpt
I0316 13:46:49.444988 140639127025472 checkpoint_load.py:85] Loaded pytorch lightning checkpoint from checkpoints/jsc-toy_classification_jsc/software/training_ckpts/best.ckpt


First we fake quantize the module to perform calibration and fine tuning - this is only required if we have INT8 calibration as currently, other precisions are not included within [pytorch-quantization](https://docs.nvidia.com/deeplearning/tensorrt/pytorch-quantization-toolkit/docs/index.html#) library.

In [4]:
!ch transform --config configs/tensorRT/jsc_toy_INT8_quantization_by_type.toml --load ../../docs/tutorials/tensorRT/checkpoints/jsc-toy_classification_jsc_2024-03-05/software/training_ckpts/best.ckpt --load-type pl

24.25s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
[2024-03-16 13:47:03,021] [INFO] [real_accelerator.py:191:get_accelerator] Setting ds_accelerator to cuda (auto detect)
usage: ch [--config PATH] [--task TASK] [--load PATH] [--load-type]
          [--batch-size NUM] [--debug] [--log-level]
          [--report-to {wandb,tensorboard}] [--seed NUM] [--quant-config TOML]
          [--training-optimizer TYPE] [--trainer-precision TYPE]
          [--learning-rate NUM] [--weight-decay NUM] [--max-epochs NUM]
          [--max-steps NUM] [--accumulate-grad-batches NUM]
          [--log-every-n-steps NUM] [--cpu NUM] [--gpu NUM] [--nodes NUM]
          [--accelerator TYPE] [--strategy TYPE] [--auto-requeue]
          [--github-ci] [--disable-dataset-cache] [--target STR]
          [--num-targets NUM] [--run-emit] [--skip-build] [--skip-test]
          [--pretrained] [--max-token-len NUM] [--project-dir DIR]
          [--project NAME] [--profile] [--no-w

In [5]:
# !ch transform --config ../../../machop/configs/tensorRT/jsc_toy_INT8_quantization_by_type.toml --load checkpoints/jsc-toy_classification_jsc/software/training_ckpts/best.ckpt --load-type pl

### Calling passes via python - to be removed from tutorial.

In [6]:
'''
./ch transform --config configs/examples/jsc_trt_quantization.toml --load ../TensorRTDev/jsc-tiny_classification_jsc_2024-03-05/software/training_ckpts/best.ckpt --load-type pl
./ch transform --config configs/examples/jsc_trt_quantization.toml --load ../TensorRTDev/jsc-trt_classification_jsc_2024-03-05/software/training_ckpts/best.ckpt --load-type pl
./ch transform --config configs/examples/jsc_trt_quantization.toml --load ../TensorRTDev/jsc-toy_classification_jsc_2024-03-05/software/training_ckpts/best.ckpt --load-type pl
./ch transform --config configs/examples/jsc_trt_quantization.toml --load ../TensorRTDev/vgg7-test-accu-0.9332.ckpt --load-type pl
'''

configs = [tensorrt_quantize_config, tensorrt_train_config, tensorrt_analysis_config]
for config in configs:
    config['batch_size'] = pass_args['batch_size']
    config['model'] = pass_args['model']
    config['data_module'] = data_module
    config['accelerator'] = 'cuda' if pass_args['accelerator'] == 'gpu' else pass_args['accelerator']
    if config['accelerator'] == 'gpu':
        os.environ['CUDA_MODULE_LOADING'] = 'LAZY'

# Restart kernel and re-run previous cells to avoid any issues
# mg_og = deepcopy_mase_graph(mg)
mg_original = deepcopy(mg.model)

# Calibrate model (passing data samples to the quantizer and deciding the best amax for activations)
mg, _ = tensorrt_fake_quantize_transform_pass(mg, pass_args=tensorrt_quantize_config)

mg, _ = tensorrt_calibrate_transform_pass(mg, pass_args=tensorrt_quantize_config)

# Conduct QAT
mg, _ = tensorrt_fine_tune_transform_pass(mg, pass_args=tensorrt_quantize_config)

# Convert and store to ONNX and then TensorRT
mg, trt_meta = tensorrt_engine_interface_pass(mg, pass_args=tensorrt_quantize_config)

INFO     Applying fake quantization to PyTorch model...
I0316 13:47:08.065116 140639127025472 utils.py:132] Applying fake quantization to PyTorch model...


INFO     Fake quantization applied to PyTorch model.
I0316 13:47:08.969787 140639127025472 utils.py:148] Fake quantization applied to PyTorch model.
INFO     Starting calibration of the model in PyTorch...
I0316 13:47:08.971959 140639127025472 calibrate.py:81] Starting calibration of the model in PyTorch...
INFO     Disabling Quantization and Enabling Calibration
I0316 13:47:08.995031 140639127025472 calibrate.py:90] Disabling Quantization and Enabling Calibration
INFO     Disabling Quantization and Enabling Calibration
I0316 13:47:08.997697 140639127025472 calibrate.py:90] Disabling Quantization and Enabling Calibration
INFO     Disabling Quantization and Enabling Calibration
I0316 13:47:09.000303 140639127025472 calibrate.py:90] Disabling Quantization and Enabling Calibration
INFO     Disabling Quantization and Enabling Calibration
I0316 13:47:09.002133 140639127025472 calibrate.py:90] Disabling Quantization and Enabling Calibration
INFO     Disabling Quantization and Enabling Calibr

TypeError: Calibrator.evaluate() missing 1 required positional argument: 'graph'

In [8]:
# Compare original tensorrt with quantized graph
trt_meta = {}
from pathlib import PosixPath

# JSC-Toy INT8 only quantization 
trt_meta['graph_path'] = PosixPath('/root/mase/mase_output/TensorRT/Quantization/TRT/2024_03_16/version_0/model.trt')

_, _ = tensorrt_analysis_pass(trt_meta['graph_path'], pass_args=tensorrt_analysis_config)

_, _ = tensorrt_analysis_pass(mg, pass_args=tensorrt_analysis_config)

INFO     
TensorRT Engine Input/Output Information:
Index | Type    | DataType | Static Shape         | Dynamic Shape        | Name
------|---------|----------|----------------------|----------------------|-----------------------
0     | Input   | FLOAT    | (256, 16)              | (256, 16)              | input
1     | Output  | FLOAT    | (256, 5)               | (256, 5)               | 109
I0316 11:19:28.639614 140089548523328 analysis.py:128] 
TensorRT Engine Input/Output Information:
Index | Type    | DataType | Static Shape         | Dynamic Shape        | Name
------|---------|----------|----------------------|----------------------|-----------------------
0     | Input   | FLOAT    | (256, 16)              | (256, 16)              | input
1     | Output  | FLOAT    | (256, 5)               | (256, 5)               | 109
INFO     Starting TensorRT transformation analysis
I0316 11:19:28.641288 140089548523328 analysis.py:202] Starting TensorRT transformation analysis


[03/16/2024-11:19:28] [TRT] [W] CUDA lazy loading is not enabled. Enabling it can significantly reduce device memory usage and speed up TensorRT initialization. See "Lazy Loading" section of CUDA documentation https://docs.nvidia.com/cuda/cuda-c-programming-guide/index.html#lazy-loading


INFO     
Results jsc-toy-quantized:
+------------------------------+---------------+
|            Metric            |     Value     |
+------------------------------+---------------+
|    Average Test Accuracy     |    0.73309    |
|      Average Precision       |    0.74935    |
|        Average Recall        |    0.73329    |
|       Average F1 Score       |    0.73688    |
|         Average Loss         |    0.7531     |
|       Average Latency        |  0.18148 ms   |
|   Average GPU Power Usage    |   64.879 W    |
| Inference Energy Consumption | 0.0032707 mWh |
+------------------------------+---------------+
I0316 11:19:32.303160 140089548523328 analysis.py:306] 
Results jsc-toy-quantized:
+------------------------------+---------------+
|            Metric            |     Value     |
+------------------------------+---------------+
|    Average Test Accuracy     |    0.73309    |
|      Average Precision       |    0.74935    |
|        Average Recall        |    0.73329    

In [6]:
# Compare original tensorrt with quantized graph
trt_meta = {}
from pathlib import PosixPath

# JSC-Toy FP16 only quantization 
trt_meta['graph_path'] = PosixPath('/root/mase/mase_output/TensorRT/Quantization/TRT/2024_03_16/version_1/model.trt')

_, _ = tensorrt_analysis_pass(trt_meta['graph_path'], pass_args=tensorrt_analysis_config)

_, _ = tensorrt_analysis_pass(mg, pass_args=tensorrt_analysis_config)

INFO     
TensorRT Engine Input/Output Information:
Index | Type    | DataType | Static Shape         | Dynamic Shape        | Name
------|---------|----------|----------------------|----------------------|-----------------------
0     | Input   | FLOAT    | (256, 16)              | (256, 16)              | input
1     | Output  | FLOAT    | (256, 5)               | (256, 5)               | 109
I0316 11:24:17.414859 140497372997440 analysis.py:128] 
TensorRT Engine Input/Output Information:
Index | Type    | DataType | Static Shape         | Dynamic Shape        | Name
------|---------|----------|----------------------|----------------------|-----------------------
0     | Input   | FLOAT    | (256, 16)              | (256, 16)              | input
1     | Output  | FLOAT    | (256, 5)               | (256, 5)               | 109
INFO     Starting TensorRT transformation analysis
I0316 11:24:17.417420 140497372997440 analysis.py:202] Starting TensorRT transformation analysis


[03/16/2024-11:24:17] [TRT] [W] CUDA lazy loading is not enabled. Enabling it can significantly reduce device memory usage and speed up TensorRT initialization. See "Lazy Loading" section of CUDA documentation https://docs.nvidia.com/cuda/cuda-c-programming-guide/index.html#lazy-loading


INFO     
Results jsc-toy-quantized:
+------------------------------+---------------+
|            Metric            |     Value     |
+------------------------------+---------------+
|    Average Test Accuracy     |    0.73325    |
|      Average Precision       |    0.74924    |
|        Average Recall        |    0.73356    |
|       Average F1 Score       |    0.73732    |
|         Average Loss         |    0.75403    |
|       Average Latency        |   0.2609 ms   |
|   Average GPU Power Usage    |   68.244 W    |
| Inference Energy Consumption | 0.0049457 mWh |
+------------------------------+---------------+
I0316 11:24:23.903356 140497372997440 analysis.py:306] 
Results jsc-toy-quantized:
+------------------------------+---------------+
|            Metric            |     Value     |
+------------------------------+---------------+
|    Average Test Accuracy     |    0.73325    |
|      Average Precision       |    0.74924    |
|        Average Recall        |    0.73356    